In [1]:
import csv
import pandas as pd

from pprint import pprint
from google.colab import drive

!pip install transformers >/dev/null

In [5]:
try:
  import models
except ImportError:
  print('Please upload models.py to you current directory')
  raise

from models import CVSSMetric

In [6]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [9]:
train_file_path = 'gdrive/MyDrive/Colab Notebooks/data/cve_train.csv'
test_file_path = 'gdrive/MyDrive/Colab Notebooks/data/cve_test.csv'

train_data = pd.read_csv(train_file_path, index_col=0)
test_data = pd.read_csv(test_file_path, index_col=0)

train_data.head(3)

,cve_id,attack_vector,attack_complexity,privileges_required,user_interaction,scope,confidentiality,integrity,availability,description,last_modified_date,published_date,base_score,exploitability_score,impact_score
50767,CVE-2019-11980,NETWORK,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,A remote code exection vulnerability was ident...,2019-06-07T00:16Z,2019-06-05T16:29Z,8.8,2.8,5.9
41253,CVE-2017-0854,NETWORK,LOW,NONE,NONE,UNCHANGED,HIGH,NONE,HIGH,An information disclosure vulnerability in the...,2019-10-03T00:03Z,2017-11-16T23:29Z,9.1,3.9,5.2
62388,CVE-2017-7595,LOCAL,LOW,NONE,REQUIRED,UNCHANGED,NONE,NONE,HIGH,The JPEGSetupEncode function in tiff_jpeg.c in...,2018-03-22T01:29Z,2017-04-09T14:59Z,5.5,1.8,3.6


In [11]:
smalltrain = train_data.iloc[:1000].copy()
smalltest = test_data.iloc[:1000].copy()
model = CVSSMetric('integrity', smalltrain, smalltest)

In [12]:
model.train(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:28.
  Batch    80  of    113.    Elapsed: 0:00:57.

  Average training loss: 1.03
  Training epoch took: 0:01:22

Running Validation...
  Accuracy: 0.45
  Validation Loss: 1.05
  Validation took: 0:00:03

======== Epoch 2 / 2 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:31.
  Batch    80  of    113.    Elapsed: 0:01:04.

  Average training loss: 1.00
  Training epoch took: 0:01:30

Running Validation...
  Accuracy: 0.45
  Validation Loss: 1.01
  Validation took: 0:00:04

Training complete!
Total training took 0:02:58 (h:mm:ss)


,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.03,1.05,0.45,0:01:22,0:00:03
2,1.00,1.01,0.45,0:01:30,0:00:04


In [13]:
model.test()

0.508

In [14]:
#print(len(model.flat_predictions))
pprint(model.getClassificationStats())

{'accuracy': 0.508,
 'cm': array([[  0,   0, 322],
       [  0,   0, 170],
       [  0,   0, 508]]),
 'f1': 0.508,
 'mcc': 0.0,
 'precision': 0.508,
 'recall': 0.508}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [15]:
model.saveModel()
model.savePredictions()